# Data Extraction

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import relevant libraries
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

# Create Consolidated DataFrame

In [ ]:
# function to create dates
from datetime import timedelta, date

def get_dates_between(start_date, end_date):
    return [start_date + timedelta(days=i)
            for i in range((end_date - start_date).days + 1)]

start_date = date(2010, 1, 1)
end_date = date(2023, 9, 30)

dates_between = get_dates_between(start_date, end_date)

# Create consolidated dataframe
eur_usd_df = pd.DataFrame({'Date': dates_between})

# convert date to datetime object
eur_usd_df['Date'] = pd.to_datetime(eur_usd_df['Date'])

eur_usd_df

,Date
0,2010-01-01
1,2010-01-02
2,2010-01-03
3,2010-01-04
4,2010-01-05
...,...
5016,2023-09-26
5017,2023-09-27
5018,2023-09-28
5019,2023-09-29


# EUR/USD Exchange Rates

retrieved EUR/USD exchange rates from yahoo finance

In [ ]:
# daily EUR/USD exchange rates
x_rates_df = pd.read_csv('/content/drive/MyDrive/QF209 Group 7/EUR_USD Historical Data.csv')
x_rates_df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-01,1.432706,1.440196,1.432706,1.438994,1.438994,0.0
1,2010-01-04,1.431004,1.445191,1.426208,1.442398,1.442398,0.0
2,2010-01-05,1.442710,1.448310,1.435194,1.436596,1.436596,0.0
3,2010-01-06,1.436596,1.443460,1.429123,1.440403,1.440403,0.0
4,2010-01-07,1.440300,1.444481,1.430206,1.431803,1.431803,0.0
...,...,...,...,...,...,...,...
3581,2023-09-25,1.064849,1.065542,1.057731,1.064849,1.064849,0.0
3582,2023-09-26,1.059165,1.060895,1.056814,1.059165,1.059165,0.0
3583,2023-09-27,1.056948,1.057373,1.050906,1.056948,1.056948,0.0
3584,2023-09-28,1.050531,1.057865,1.049186,1.050531,1.050531,0.0


In [ ]:
# check for null values and data type
x_rates_df.isnull().sum()

Date         0
Open         5
High         5
Low          5
Close        5
Adj Close    5
Volume       5
dtype: int64

In [ ]:
# convert date to datetime object
x_rates_df['Date'] = pd.to_datetime(x_rates_df['Date'])
x_rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3586 entries, 0 to 3585
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       3586 non-null   datetime64[ns]
 1   Open       3581 non-null   float64       
 2   High       3581 non-null   float64       
 3   Low        3581 non-null   float64       
 4   Close      3581 non-null   float64       
 5   Adj Close  3581 non-null   float64       
 6   Volume     3581 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 196.2 KB


In [ ]:
# check null values
# x_rates_df[x_rates_df.isna().any(axis=1)]
x_rates_df.isnull().sum()

Date         0
Open         5
High         5
Low          5
Close        5
Adj Close    5
Volume       5
dtype: int64

In [ ]:
# remove rows with at least one missing value
x_rates_df = x_rates_df.dropna()
x_rates_df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
# check volume column
x_rates_df['Volume'].value_counts()

0.0    3581
Name: Volume, dtype: int64

In [ ]:
# remove volume column
x_rates_df = x_rates_df[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close']]

# add to consolidated dataframe
eur_usd_df = eur_usd_df.merge(x_rates_df, how='left', on='Date')
eur_usd_df

,Date,Open,High,Low,Close,Adj Close
0,2010-01-01,1.432706,1.440196,1.432706,1.438994,1.438994
1,2010-01-02,NaN,NaN,NaN,NaN,NaN
2,2010-01-03,NaN,NaN,NaN,NaN,NaN
3,2010-01-04,1.431004,1.445191,1.426208,1.442398,1.442398
4,2010-01-05,1.442710,1.448310,1.435194,1.436596,1.436596
...,...,...,...,...,...,...
5016,2023-09-26,1.059165,1.060895,1.056814,1.059165,1.059165
5017,2023-09-27,1.056948,1.057373,1.050906,1.056948,1.056948
5018,2023-09-28,1.050531,1.057865,1.049186,1.050531,1.050531
5019,2023-09-29,1.056245,1.061797,1.055855,1.056245,1.056245


# Interest Rates

## European Central Bank

retrieved daily ECV interest rates from ECB Data Portal

In [ ]:
# read csv file
eur_int_rates_df = pd.read_csv('/content/drive/MyDrive/QF209 Group 7/ECB Interest Rates.csv')
eur_int_rates_df

,DATE,TIME PERIOD,ECB Marginal lending facility - date of changes (raw data) - Level (FM.D.U2.EUR.4F.KR.MLFR.LEV)
0,1999-01-01,01 Jan 1999,4.50
1,1999-01-02,02 Jan 1999,4.50
2,1999-01-03,03 Jan 1999,4.50
3,1999-01-04,04 Jan 1999,3.25
4,1999-01-05,05 Jan 1999,3.25
...,...,...,...
9043,2023-10-05,05 Oct 2023,4.75
9044,2023-10-06,06 Oct 2023,4.75
9045,2023-10-07,07 Oct 2023,4.75
9046,2023-10-08,08 Oct 2023,4.75


In [ ]:
# check for null values and datatype
# eur_int_rates_df.info()
eur_int_rates_df.isnull().sum()

DATE                                                                                               0
TIME PERIOD                                                                                        0
ECB Marginal lending facility - date of changes (raw data) - Level (FM.D.U2.EUR.4F.KR.MLFR.LEV)    0
dtype: int64

In [ ]:
# drop time period column
eur_int_rates_df = eur_int_rates_df.drop('TIME PERIOD', axis=1)
eur_int_rates_df.head()

,DATE,ECB Marginal lending facility - date of changes (raw data) - Level (FM.D.U2.EUR.4F.KR.MLFR.LEV)
0,1999-01-01,4.50
1,1999-01-02,4.50
2,1999-01-03,4.50
3,1999-01-04,3.25
4,1999-01-05,3.25


In [ ]:
# rename columns
eur_int_rates_df = eur_int_rates_df.rename(columns={"DATE": "Date", "ECB Marginal lending facility - date of changes (raw data) - Level (FM.D.U2.EUR.4F.KR.MLFR.LEV)": "EUR I/R"})
eur_int_rates_df.head()

,Date,EUR I/R
0,1999-01-01,4.50
1,1999-01-02,4.50
2,1999-01-03,4.50
3,1999-01-04,3.25
4,1999-01-05,3.25


In [ ]:
# convert date to datetime object
eur_int_rates_df['Date'] = pd.to_datetime(eur_int_rates_df['Date'])
eur_int_rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9048 entries, 0 to 9047
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     9048 non-null   datetime64[ns]
 1   EUR I/R  9048 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 141.5 KB


In [ ]:
# add to consolidated dataframe
eur_usd_df = eur_usd_df.merge(eur_int_rates_df, how='left', on='Date')
eur_usd_df

,Date,Open,High,Low,Close,Adj Close,EUR I/R
0,2010-01-01,1.432706,1.440196,1.432706,1.438994,1.438994,1.75
1,2010-01-02,NaN,NaN,NaN,NaN,NaN,1.75
2,2010-01-03,NaN,NaN,NaN,NaN,NaN,1.75
3,2010-01-04,1.431004,1.445191,1.426208,1.442398,1.442398,1.75
4,2010-01-05,1.442710,1.448310,1.435194,1.436596,1.436596,1.75
...,...,...,...,...,...,...,...
5016,2023-09-26,1.059165,1.060895,1.056814,1.059165,1.059165,4.75
5017,2023-09-27,1.056948,1.057373,1.050906,1.056948,1.056948,4.75
5018,2023-09-28,1.050531,1.057865,1.049186,1.050531,1.050531,4.75
5019,2023-09-29,1.056245,1.061797,1.055855,1.056245,1.056245,4.75


In [ ]:
# check info of updated dataframe
eur_usd_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5021 entries, 0 to 5020
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       5021 non-null   datetime64[ns]
 1   Open       3581 non-null   float64       
 2   High       3581 non-null   float64       
 3   Low        3581 non-null   float64       
 4   Close      3581 non-null   float64       
 5   Adj Close  3581 non-null   float64       
 6   EUR I/R    5021 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 313.8 KB


## US Federal Reserve

retrieved csv file on weekly US Fed Rates (mean, ending Friday) from FRED economic data

In [ ]:
# read csv file
us_fed_rates_df = pd.read_csv('/content/drive/MyDrive/QF209 Group 7/US Fed Rates.csv')
us_fed_rates_df

,DATE,DFF
0,2010-01-01,0.095714
1,2010-01-08,0.095714
2,2010-01-15,0.111429
3,2010-01-22,0.121429
4,2010-01-29,0.117143
...,...,...
714,2023-09-08,5.330000
715,2023-09-15,5.330000
716,2023-09-22,5.330000
717,2023-09-29,5.330000


In [ ]:
# check for null values and datatype
us_fed_rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719 entries, 0 to 718
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DATE    719 non-null    object 
 1   DFF     719 non-null    float64
dtypes: float64(1), object(1)
memory usage: 11.4+ KB


In [ ]:
# change column names
us_fed_rates_df = us_fed_rates_df.rename(columns={"DATE": "Date", 'DFF': 'US Fed Rate'})
us_fed_rates_df.head()

,Date,US Fed Rate
0,2010-01-01,0.095714
1,2010-01-08,0.095714
2,2010-01-15,0.111429
3,2010-01-22,0.121429
4,2010-01-29,0.117143


In [ ]:
# change date to datetime object
us_fed_rates_df['Date'] = pd.to_datetime(us_fed_rates_df['Date'])
us_fed_rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 719 entries, 0 to 718
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         719 non-null    datetime64[ns]
 1   US Fed Rate  719 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 11.4 KB


In [ ]:
# add to consolidated dataframe
eur_usd_df = eur_usd_df.merge(us_fed_rates_df, how='left', on='Date')
eur_usd_df

,Date,Open,High,Low,Close,Adj Close,EUR I/R,US Fed Rate
0,2010-01-01,1.432706,1.440196,1.432706,1.438994,1.438994,1.75,0.095714
1,2010-01-02,NaN,NaN,NaN,NaN,NaN,1.75,NaN
2,2010-01-03,NaN,NaN,NaN,NaN,NaN,1.75,NaN
3,2010-01-04,1.431004,1.445191,1.426208,1.442398,1.442398,1.75,NaN
4,2010-01-05,1.442710,1.448310,1.435194,1.436596,1.436596,1.75,NaN
...,...,...,...,...,...,...,...,...
5016,2023-09-26,1.059165,1.060895,1.056814,1.059165,1.059165,4.75,NaN
5017,2023-09-27,1.056948,1.057373,1.050906,1.056948,1.056948,4.75,NaN
5018,2023-09-28,1.050531,1.057865,1.049186,1.050531,1.050531,4.75,NaN
5019,2023-09-29,1.056245,1.061797,1.055855,1.056245,1.056245,4.75,5.330000


In [ ]:
# backward fill fed rates
eur_usd_df['US Fed Rate'] = eur_usd_df['US Fed Rate'].bfill()
eur_usd_df

,Date,Open,High,Low,Close,Adj Close,EUR I/R,US Fed Rate
0,2010-01-01,1.432706,1.440196,1.432706,1.438994,1.438994,1.75,0.095714
1,2010-01-02,NaN,NaN,NaN,NaN,NaN,1.75,0.095714
2,2010-01-03,NaN,NaN,NaN,NaN,NaN,1.75,0.095714
3,2010-01-04,1.431004,1.445191,1.426208,1.442398,1.442398,1.75,0.095714
4,2010-01-05,1.442710,1.448310,1.435194,1.436596,1.436596,1.75,0.095714
...,...,...,...,...,...,...,...,...
5016,2023-09-26,1.059165,1.060895,1.056814,1.059165,1.059165,4.75,5.330000
5017,2023-09-27,1.056948,1.057373,1.050906,1.056948,1.056948,4.75,5.330000
5018,2023-09-28,1.050531,1.057865,1.049186,1.050531,1.050531,4.75,5.330000
5019,2023-09-29,1.056245,1.061797,1.055855,1.056245,1.056245,4.75,5.330000


In [ ]:
# check success of backfill
eur_usd_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5021 entries, 0 to 5020
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         5021 non-null   datetime64[ns]
 1   Open         3581 non-null   float64       
 2   High         3581 non-null   float64       
 3   Low          3581 non-null   float64       
 4   Close        3581 non-null   float64       
 5   Adj Close    3581 non-null   float64       
 6   EUR I/R      5021 non-null   float64       
 7   US Fed Rate  5020 non-null   float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 353.0 KB


# Inflation Rates

Headline: All goods and services <br/>
Core: All goods and services minus food and energy prices

## US Headline CPI

retrieved csv file on monthly headline CPI values (start of month) from FRED Economic Data

In [ ]:
# read csv file
us_head_cpi_df = pd.read_csv('/content/drive/MyDrive/QF209 Group 7/US Headline Inflation Data.csv')
us_head_cpi_df

,DATE,CPIAUCSL
0,2010-01-01,217.488
1,2010-02-01,217.281
2,2010-03-01,217.353
3,2010-04-01,217.403
4,2010-05-01,217.290
...,...,...
159,2023-04-01,302.918
160,2023-05-01,303.294
161,2023-06-01,303.841
162,2023-07-01,304.348


In [ ]:
# understanding dataframe
us_head_cpi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DATE      164 non-null    object 
 1   CPIAUCSL  164 non-null    float64
dtypes: float64(1), object(1)
memory usage: 2.7+ KB


In [ ]:
# rename columns
us_head_cpi_df = us_head_cpi_df.rename(columns={"DATE": "Date", 'CPIAUCSL': 'US Headline CPI'})
us_head_cpi_df

,Date,US Headline CPI
0,2010-01-01,217.488
1,2010-02-01,217.281
2,2010-03-01,217.353
3,2010-04-01,217.403
4,2010-05-01,217.290
...,...,...
159,2023-04-01,302.918
160,2023-05-01,303.294
161,2023-06-01,303.841
162,2023-07-01,304.348


In [ ]:
# change date to datetime object
us_head_cpi_df['Date'] = pd.to_datetime(us_head_cpi_df['Date'])
us_head_cpi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             164 non-null    datetime64[ns]
 1   US Headline CPI  164 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.7 KB


In [ ]:
# add to consolidated dataframe
eur_usd_df = eur_usd_df.merge(us_head_cpi_df, how='left', on='Date')
eur_usd_df

,Date,Open,High,Low,Close,Adj Close,EUR I/R,US Fed Rate,US Headline CPI
0,2010-01-01,1.432706,1.440196,1.432706,1.438994,1.438994,1.75,0.095714,217.488
1,2010-01-02,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,NaN
2,2010-01-03,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,NaN
3,2010-01-04,1.431004,1.445191,1.426208,1.442398,1.442398,1.75,0.095714,NaN
4,2010-01-05,1.442710,1.448310,1.435194,1.436596,1.436596,1.75,0.095714,NaN
...,...,...,...,...,...,...,...,...,...
5016,2023-09-26,1.059165,1.060895,1.056814,1.059165,1.059165,4.75,5.330000,NaN
5017,2023-09-27,1.056948,1.057373,1.050906,1.056948,1.056948,4.75,5.330000,NaN
5018,2023-09-28,1.050531,1.057865,1.049186,1.050531,1.050531,4.75,5.330000,NaN
5019,2023-09-29,1.056245,1.061797,1.055855,1.056245,1.056245,4.75,5.330000,NaN


In [ ]:
# forward fill US Headline CPI
eur_usd_df['US Headline CPI'] = eur_usd_df['US Headline CPI'].ffill()
eur_usd_df

,Date,Open,High,Low,Close,Adj Close,EUR I/R,US Fed Rate,US Headline CPI
0,2010-01-01,1.432706,1.440196,1.432706,1.438994,1.438994,1.75,0.095714,217.488
1,2010-01-02,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488
2,2010-01-03,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488
3,2010-01-04,1.431004,1.445191,1.426208,1.442398,1.442398,1.75,0.095714,217.488
4,2010-01-05,1.442710,1.448310,1.435194,1.436596,1.436596,1.75,0.095714,217.488
...,...,...,...,...,...,...,...,...,...
5016,2023-09-26,1.059165,1.060895,1.056814,1.059165,1.059165,4.75,5.330000,306.269
5017,2023-09-27,1.056948,1.057373,1.050906,1.056948,1.056948,4.75,5.330000,306.269
5018,2023-09-28,1.050531,1.057865,1.049186,1.050531,1.050531,4.75,5.330000,306.269
5019,2023-09-29,1.056245,1.061797,1.055855,1.056245,1.056245,4.75,5.330000,306.269


## US Core CPI

retrieved csv file on monthly core CPI values (start of month) from FRED Economic Data

In [ ]:
# read csv file
us_core_cpi_df = pd.read_csv('/content/drive/MyDrive/QF209 Group 7/US Core Inflation Data.csv')
us_core_cpi_df

,DATE,CPILFESL
0,2010-01-01,220.633
1,2010-02-01,220.731
2,2010-03-01,220.783
3,2010-04-01,220.822
4,2010-05-01,220.962
...,...,...
159,2023-04-01,306.489
160,2023-05-01,307.824
161,2023-06-01,308.309
162,2023-07-01,308.801


In [ ]:
# understanding dataframe
us_core_cpi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DATE      164 non-null    object 
 1   CPILFESL  164 non-null    float64
dtypes: float64(1), object(1)
memory usage: 2.7+ KB


In [ ]:
# rename columns
us_core_cpi_df = us_core_cpi_df.rename(columns={"DATE": "Date", 'CPILFESL': 'US Core CPI'})
us_core_cpi_df.head()

,Date,US Core CPI
0,2010-01-01,220.633
1,2010-02-01,220.731
2,2010-03-01,220.783
3,2010-04-01,220.822
4,2010-05-01,220.962


In [ ]:
# change date to datetime object
us_core_cpi_df['Date'] = pd.to_datetime(us_core_cpi_df['Date'])
us_core_cpi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         164 non-null    datetime64[ns]
 1   US Core CPI  164 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.7 KB


In [ ]:
# add to consolidated dataframe
eur_usd_df = eur_usd_df.merge(us_core_cpi_df, how='left', on='Date')
eur_usd_df

,Date,Open,High,Low,Close,Adj Close,EUR I/R,US Fed Rate,US Headline CPI,US Core CPI
0,2010-01-01,1.432706,1.440196,1.432706,1.438994,1.438994,1.75,0.095714,217.488,220.633
1,2010-01-02,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488,NaN
2,2010-01-03,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488,NaN
3,2010-01-04,1.431004,1.445191,1.426208,1.442398,1.442398,1.75,0.095714,217.488,NaN
4,2010-01-05,1.442710,1.448310,1.435194,1.436596,1.436596,1.75,0.095714,217.488,NaN
...,...,...,...,...,...,...,...,...,...,...
5016,2023-09-26,1.059165,1.060895,1.056814,1.059165,1.059165,4.75,5.330000,306.269,NaN
5017,2023-09-27,1.056948,1.057373,1.050906,1.056948,1.056948,4.75,5.330000,306.269,NaN
5018,2023-09-28,1.050531,1.057865,1.049186,1.050531,1.050531,4.75,5.330000,306.269,NaN
5019,2023-09-29,1.056245,1.061797,1.055855,1.056245,1.056245,4.75,5.330000,306.269,NaN


In [ ]:
# forward fill CPI
eur_usd_df['US Core CPI'] = eur_usd_df['US Core CPI'].ffill()
eur_usd_df

,Date,Open,High,Low,Close,Adj Close,EUR I/R,US Fed Rate,US Headline CPI,US Core CPI
0,2010-01-01,1.432706,1.440196,1.432706,1.438994,1.438994,1.75,0.095714,217.488,220.633
1,2010-01-02,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488,220.633
2,2010-01-03,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488,220.633
3,2010-01-04,1.431004,1.445191,1.426208,1.442398,1.442398,1.75,0.095714,217.488,220.633
4,2010-01-05,1.442710,1.448310,1.435194,1.436596,1.436596,1.75,0.095714,217.488,220.633
...,...,...,...,...,...,...,...,...,...,...
5016,2023-09-26,1.059165,1.060895,1.056814,1.059165,1.059165,4.75,5.330000,306.269,309.661
5017,2023-09-27,1.056948,1.057373,1.050906,1.056948,1.056948,4.75,5.330000,306.269,309.661
5018,2023-09-28,1.050531,1.057865,1.049186,1.050531,1.050531,4.75,5.330000,306.269,309.661
5019,2023-09-29,1.056245,1.061797,1.055855,1.056245,1.056245,4.75,5.330000,306.269,309.661


## EUR Headline Inflation

retrieved csv file on monthly headline CPI values (end of month) from ECB Economic Data

In [ ]:
# read csv file
eu_headline_cpi_df = pd.read_csv('/content/drive/MyDrive/QF209 Group 7/EUR Headline Inflation Data.csv')
eu_headline_cpi_df

,DATE,TIME PERIOD,HICP - Overall index (ICP.M.U2.N.000000.4.INX)
0,1996-01-31,1996Jan,70.97
1,1996-02-29,1996Feb,71.29
2,1996-03-31,1996Mar,71.54
3,1996-04-30,1996Apr,71.66
4,1996-05-31,1996May,71.83
...,...,...,...
328,2023-05-31,2023May,123.15
329,2023-06-30,2023Jun,123.47
330,2023-07-31,2023Jul,123.36
331,2023-08-31,2023Aug,124.03


In [ ]:
# understanding dataframe
eu_headline_cpi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 3 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   DATE                                            333 non-null    object 
 1   TIME PERIOD                                     333 non-null    object 
 2   HICP - Overall index (ICP.M.U2.N.000000.4.INX)  333 non-null    float64
dtypes: float64(1), object(2)
memory usage: 7.9+ KB


In [ ]:
# drop TIME PERIOD column
eu_headline_cpi_df = eu_headline_cpi_df.drop('TIME PERIOD', axis=1)
eu_headline_cpi_df.head()

,DATE,HICP - Overall index (ICP.M.U2.N.000000.4.INX)
0,1996-01-31,70.97
1,1996-02-29,71.29
2,1996-03-31,71.54
3,1996-04-30,71.66
4,1996-05-31,71.83


In [ ]:
# rename columns
eu_headline_cpi_df = eu_headline_cpi_df.rename(columns={"DATE": "Date", 'HICP - Overall index (ICP.M.U2.N.000000.4.INX)': 'EUR Headline CPI'})
eu_headline_cpi_df.head()

,Date,EUR Headline CPI
0,1996-01-31,70.97
1,1996-02-29,71.29
2,1996-03-31,71.54
3,1996-04-30,71.66
4,1996-05-31,71.83


In [ ]:
# change date to datetime object
eu_headline_cpi_df['Date'] = pd.to_datetime(eu_headline_cpi_df['Date'])
eu_headline_cpi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              333 non-null    datetime64[ns]
 1   EUR Headline CPI  333 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 5.3 KB


In [ ]:
# add to consolidated dataframe
eur_usd_df = eur_usd_df.merge(eu_headline_cpi_df, how='left', on='Date')
eur_usd_df

,Date,Open,High,Low,Close,Adj Close,EUR I/R,US Fed Rate,US Headline CPI,US Core CPI,EUR Headline CPI
0,2010-01-01,1.432706,1.440196,1.432706,1.438994,1.438994,1.75,0.095714,217.488,220.633,NaN
1,2010-01-02,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488,220.633,NaN
2,2010-01-03,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488,220.633,NaN
3,2010-01-04,1.431004,1.445191,1.426208,1.442398,1.442398,1.75,0.095714,217.488,220.633,NaN
4,2010-01-05,1.442710,1.448310,1.435194,1.436596,1.436596,1.75,0.095714,217.488,220.633,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5016,2023-09-26,1.059165,1.060895,1.056814,1.059165,1.059165,4.75,5.330000,306.269,309.661,NaN
5017,2023-09-27,1.056948,1.057373,1.050906,1.056948,1.056948,4.75,5.330000,306.269,309.661,NaN
5018,2023-09-28,1.050531,1.057865,1.049186,1.050531,1.050531,4.75,5.330000,306.269,309.661,NaN
5019,2023-09-29,1.056245,1.061797,1.055855,1.056245,1.056245,4.75,5.330000,306.269,309.661,NaN


In [ ]:
# retrieve EUR Headline CPI in Dec 2009
eu_dec2019_headline_cpi = eu_headline_cpi_df.loc[(eu_headline_cpi_df['Date'].dt.month == 12) & (eu_headline_cpi_df['Date'].dt.year == 2009)]['EUR Headline CPI']

# set 2010-01-01 EUR Headline CPI data with EUR Headline CPI in Dec 2009
eur_usd_df['EUR Headline CPI'].iloc[0] = eu_dec2019_headline_cpi

# forward fill CPI
eur_usd_df['EUR Headline CPI'] = eur_usd_df['EUR Headline CPI'].ffill()
eur_usd_df

,Date,Open,High,Low,Close,Adj Close,EUR I/R,US Fed Rate,US Headline CPI,US Core CPI,EUR Headline CPI
0,2010-01-01,1.432706,1.440196,1.432706,1.438994,1.438994,1.75,0.095714,217.488,220.633,92.32
1,2010-01-02,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488,220.633,92.32
2,2010-01-03,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488,220.633,92.32
3,2010-01-04,1.431004,1.445191,1.426208,1.442398,1.442398,1.75,0.095714,217.488,220.633,92.32
4,2010-01-05,1.442710,1.448310,1.435194,1.436596,1.436596,1.75,0.095714,217.488,220.633,92.32
...,...,...,...,...,...,...,...,...,...,...,...
5016,2023-09-26,1.059165,1.060895,1.056814,1.059165,1.059165,4.75,5.330000,306.269,309.661,124.03
5017,2023-09-27,1.056948,1.057373,1.050906,1.056948,1.056948,4.75,5.330000,306.269,309.661,124.03
5018,2023-09-28,1.050531,1.057865,1.049186,1.050531,1.050531,4.75,5.330000,306.269,309.661,124.03
5019,2023-09-29,1.056245,1.061797,1.055855,1.056245,1.056245,4.75,5.330000,306.269,309.661,124.03


## EUR Core Inflation

retrieved csv file on monthly headline CPI values (end of month) from ECB Economic Data

In [ ]:
# read csv file
eu_core_cpi_df = pd.read_csv('/content/drive/MyDrive/QF209 Group 7/EUR Core Inflation Data.csv')
eu_core_cpi_df

,DATE,TIME PERIOD,HICP - All-items excluding energy and seasonal food (ICP.M.U2.N.XESEAS.4.INX)
0,1996-01-31,1996Jan,73.77
1,1996-02-29,1996Feb,74.08
2,1996-03-31,1996Mar,74.27
3,1996-04-30,1996Apr,74.32
4,1996-05-31,1996May,74.48
...,...,...,...
327,2023-04-30,2023Apr,119.15
328,2023-05-31,2023May,119.48
329,2023-06-30,2023Jun,119.91
330,2023-07-31,2023Jul,119.86


In [ ]:
# understanding dataframe
eu_core_cpi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332 entries, 0 to 331
Data columns (total 3 columns):
 #   Column                                                                         Non-Null Count  Dtype  
---  ------                                                                         --------------  -----  
 0   DATE                                                                           332 non-null    object 
 1   TIME PERIOD                                                                    332 non-null    object 
 2   HICP - All-items excluding energy and seasonal food (ICP.M.U2.N.XESEAS.4.INX)  332 non-null    float64
dtypes: float64(1), object(2)
memory usage: 7.9+ KB


In [ ]:
# drop TIME PERIOD column
eu_core_cpi_df = eu_core_cpi_df.drop('TIME PERIOD', axis=1)
eu_core_cpi_df.head()

,DATE,HICP - All-items excluding energy and seasonal food (ICP.M.U2.N.XESEAS.4.INX)
0,1996-01-31,73.77
1,1996-02-29,74.08
2,1996-03-31,74.27
3,1996-04-30,74.32
4,1996-05-31,74.48


In [ ]:
# rename columns
eu_core_cpi_df = eu_core_cpi_df.rename(columns={"DATE": "Date", 'HICP - All-items excluding energy and seasonal food (ICP.M.U2.N.XESEAS.4.INX)': 'EUR Core CPI'})
eu_core_cpi_df.head()

,Date,EUR Core CPI
0,1996-01-31,73.77
1,1996-02-29,74.08
2,1996-03-31,74.27
3,1996-04-30,74.32
4,1996-05-31,74.48


In [ ]:
# change date to datetime object
eu_core_cpi_df['Date'] = pd.to_datetime(eu_core_cpi_df['Date'])
eu_core_cpi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332 entries, 0 to 331
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          332 non-null    datetime64[ns]
 1   EUR Core CPI  332 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 5.3 KB


In [ ]:
# add to consolidated dataframe
eur_usd_df = eur_usd_df.merge(eu_core_cpi_df, how='left', on='Date')
eur_usd_df

,Date,Open,High,Low,Close,Adj Close,EUR I/R,US Fed Rate,US Headline CPI,US Core CPI,EUR Headline CPI,EUR Core CPI
0,2010-01-01,1.432706,1.440196,1.432706,1.438994,1.438994,1.75,0.095714,217.488,220.633,92.32,NaN
1,2010-01-02,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488,220.633,92.32,NaN
2,2010-01-03,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488,220.633,92.32,NaN
3,2010-01-04,1.431004,1.445191,1.426208,1.442398,1.442398,1.75,0.095714,217.488,220.633,92.32,NaN
4,2010-01-05,1.442710,1.448310,1.435194,1.436596,1.436596,1.75,0.095714,217.488,220.633,92.32,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5016,2023-09-26,1.059165,1.060895,1.056814,1.059165,1.059165,4.75,5.330000,306.269,309.661,124.03,NaN
5017,2023-09-27,1.056948,1.057373,1.050906,1.056948,1.056948,4.75,5.330000,306.269,309.661,124.03,NaN
5018,2023-09-28,1.050531,1.057865,1.049186,1.050531,1.050531,4.75,5.330000,306.269,309.661,124.03,NaN
5019,2023-09-29,1.056245,1.061797,1.055855,1.056245,1.056245,4.75,5.330000,306.269,309.661,124.03,NaN


In [ ]:
# retrieve EUR Headline CPI in Dec 2009
eu_dec2019_core_cpi = eu_core_cpi_df.loc[(eu_core_cpi_df['Date'].dt.month == 12) & (eu_core_cpi_df['Date'].dt.year == 2009)]['EUR Core CPI']

# set 2010-01-01 EUR Headline CPI data with EUR Headline CPI in Dec 2009
eur_usd_df['EUR Core CPI'].iloc[0] = eu_dec2019_core_cpi

# forward fill CPI
eur_usd_df['EUR Core CPI'] = eur_usd_df['EUR Core CPI'].ffill()
eur_usd_df

,Date,Open,High,Low,Close,Adj Close,EUR I/R,US Fed Rate,US Headline CPI,US Core CPI,EUR Headline CPI,EUR Core CPI
0,2010-01-01,1.432706,1.440196,1.432706,1.438994,1.438994,1.75,0.095714,217.488,220.633,92.32,93.52
1,2010-01-02,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488,220.633,92.32,93.52
2,2010-01-03,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488,220.633,92.32,93.52
3,2010-01-04,1.431004,1.445191,1.426208,1.442398,1.442398,1.75,0.095714,217.488,220.633,92.32,93.52
4,2010-01-05,1.442710,1.448310,1.435194,1.436596,1.436596,1.75,0.095714,217.488,220.633,92.32,93.52
...,...,...,...,...,...,...,...,...,...,...,...,...
5016,2023-09-26,1.059165,1.060895,1.056814,1.059165,1.059165,4.75,5.330000,306.269,309.661,124.03,120.23
5017,2023-09-27,1.056948,1.057373,1.050906,1.056948,1.056948,4.75,5.330000,306.269,309.661,124.03,120.23
5018,2023-09-28,1.050531,1.057865,1.049186,1.050531,1.050531,4.75,5.330000,306.269,309.661,124.03,120.23
5019,2023-09-29,1.056245,1.061797,1.055855,1.056245,1.056245,4.75,5.330000,306.269,309.661,124.03,120.23


# Gross Domestic Product (GDP)

## US GDP Per Capita

retrieved quarterly csv file from FRED Economic Data <br>

In [ ]:
# read csv file
us_gdp_per_capita = pd.read_csv('/content/drive/MyDrive/QF209 Group 7/US GDP Per Capita Data.csv')
us_gdp_per_capita.head()

,DATE,A939RC0Q052SBEA
0,2010-01-01,47797.0
1,2010-04-01,48403.0
2,2010-07-01,48821.0
3,2010-10-01,49256.0
4,2011-01-01,49302.0


In [ ]:
# understand dataframe
us_gdp_per_capita.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   DATE             54 non-null     object 
 1   A939RC0Q052SBEA  54 non-null     float64
dtypes: float64(1), object(1)
memory usage: 992.0+ bytes


In [ ]:
# rename columns
us_gdp_per_capita = us_gdp_per_capita.rename(columns={"DATE": "Date", 'A939RC0Q052SBEA': 'US GDP Per Capita'})
us_gdp_per_capita.head()

,Date,US GDP Per Capita
0,2010-01-01,47797.0
1,2010-04-01,48403.0
2,2010-07-01,48821.0
3,2010-10-01,49256.0
4,2011-01-01,49302.0


In [ ]:
# change date to datetime object
us_gdp_per_capita['Date'] = pd.to_datetime(us_gdp_per_capita['Date'])
us_gdp_per_capita.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               54 non-null     datetime64[ns]
 1   US GDP Per Capita  54 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 992.0 bytes


In [ ]:
# add to consolidated dataframe
eur_usd_df = eur_usd_df.merge(us_gdp_per_capita, how='left', on='Date')
eur_usd_df

,Date,Open,High,Low,Close,Adj Close,EUR I/R,US Fed Rate,US Headline CPI,US Core CPI,EUR Headline CPI,EUR Core CPI,US GDP Per Capita
0,2010-01-01,1.432706,1.440196,1.432706,1.438994,1.438994,1.75,0.095714,217.488,220.633,92.32,93.52,47797.0
1,2010-01-02,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488,220.633,92.32,93.52,NaN
2,2010-01-03,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488,220.633,92.32,93.52,NaN
3,2010-01-04,1.431004,1.445191,1.426208,1.442398,1.442398,1.75,0.095714,217.488,220.633,92.32,93.52,NaN
4,2010-01-05,1.442710,1.448310,1.435194,1.436596,1.436596,1.75,0.095714,217.488,220.633,92.32,93.52,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5016,2023-09-26,1.059165,1.060895,1.056814,1.059165,1.059165,4.75,5.330000,306.269,309.661,124.03,120.23,NaN
5017,2023-09-27,1.056948,1.057373,1.050906,1.056948,1.056948,4.75,5.330000,306.269,309.661,124.03,120.23,NaN
5018,2023-09-28,1.050531,1.057865,1.049186,1.050531,1.050531,4.75,5.330000,306.269,309.661,124.03,120.23,NaN
5019,2023-09-29,1.056245,1.061797,1.055855,1.056245,1.056245,4.75,5.330000,306.269,309.661,124.03,120.23,NaN


In [ ]:
# forward fill US GDP Per Capita values
eur_usd_df['US GDP Per Capita'] = eur_usd_df['US GDP Per Capita'].ffill()
eur_usd_df

,Date,Open,High,Low,Close,Adj Close,EUR I/R,US Fed Rate,US Headline CPI,US Core CPI,EUR Headline CPI,EUR Core CPI,US GDP Per Capita
0,2010-01-01,1.432706,1.440196,1.432706,1.438994,1.438994,1.75,0.095714,217.488,220.633,92.32,93.52,47797.0
1,2010-01-02,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488,220.633,92.32,93.52,47797.0
2,2010-01-03,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488,220.633,92.32,93.52,47797.0
3,2010-01-04,1.431004,1.445191,1.426208,1.442398,1.442398,1.75,0.095714,217.488,220.633,92.32,93.52,47797.0
4,2010-01-05,1.442710,1.448310,1.435194,1.436596,1.436596,1.75,0.095714,217.488,220.633,92.32,93.52,47797.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5016,2023-09-26,1.059165,1.060895,1.056814,1.059165,1.059165,4.75,5.330000,306.269,309.661,124.03,120.23,80781.0
5017,2023-09-27,1.056948,1.057373,1.050906,1.056948,1.056948,4.75,5.330000,306.269,309.661,124.03,120.23,80781.0
5018,2023-09-28,1.050531,1.057865,1.049186,1.050531,1.050531,4.75,5.330000,306.269,309.661,124.03,120.23,80781.0
5019,2023-09-29,1.056245,1.061797,1.055855,1.056245,1.056245,4.75,5.330000,306.269,309.661,124.03,120.23,80781.0


In [ ]:
# remove Jul 2023 and Aug 2023 as data is quarterly
eur_usd_df = eur_usd_df.loc[(eur_usd_df['Date'].dt.date < pd.to_datetime('2023-07-01'))]
eur_usd_df

,Date,Open,High,Low,Close,Adj Close,EUR I/R,US Fed Rate,US Headline CPI,US Core CPI,EUR Headline CPI,EUR Core CPI,US GDP Per Capita
0,2010-01-01,1.432706,1.440196,1.432706,1.438994,1.438994,1.75,0.095714,217.488,220.633,92.32,93.52,47797.0
1,2010-01-02,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488,220.633,92.32,93.52,47797.0
2,2010-01-03,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488,220.633,92.32,93.52,47797.0
3,2010-01-04,1.431004,1.445191,1.426208,1.442398,1.442398,1.75,0.095714,217.488,220.633,92.32,93.52,47797.0
4,2010-01-05,1.442710,1.448310,1.435194,1.436596,1.436596,1.75,0.095714,217.488,220.633,92.32,93.52,47797.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4924,2023-06-26,1.090667,1.092180,1.088815,1.090667,1.090667,4.25,5.071429,303.841,308.309,123.15,119.48,80781.0
4925,2023-06-27,1.091048,1.097610,1.090322,1.091048,1.091048,4.25,5.071429,303.841,308.309,123.15,119.48,80781.0
4926,2023-06-28,1.095722,1.096251,1.089966,1.095722,1.095722,4.25,5.071429,303.841,308.309,123.15,119.48,80781.0
4927,2023-06-29,1.091584,1.094068,1.086236,1.091584,1.091584,4.25,5.071429,303.841,308.309,123.15,119.48,80781.0


## EUR GDP Per Capita

In [ ]:
# read csv file
eu_gdp_per_capita = pd.read_csv('/content/drive/MyDrive/QF209 Group 7/EUR GDP data.csv')
eu_gdp_per_capita.head()

,DATE,Gross domestic product at market prices
0,3/31/1995,1340390.05
1,6/30/1995,1385541.37
2,9/30/1995,1386333.58
3,12/31/1995,1471601.75
4,3/31/1996,1402186.87


In [ ]:
# understand dataframe
eu_gdp_per_capita.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 2 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   DATE                                     114 non-null    object 
 1   Gross domestic product at market prices  114 non-null    float64
dtypes: float64(1), object(1)
memory usage: 1.9+ KB


In [ ]:
# rename columns
eu_gdp_per_capita = eu_gdp_per_capita.rename(columns={"DATE": "Date", 'Gross domestic product at market prices': 'EUR GDP Per Capita'})
eu_gdp_per_capita

,Date,EUR GDP Per Capita
0,3/31/1995,1340390.05
1,6/30/1995,1385541.37
2,9/30/1995,1386333.58
3,12/31/1995,1471601.75
4,3/31/1996,1402186.87
...,...,...
109,6/30/2022,3313699.00
110,9/30/2022,3348600.79
111,12/31/2022,3515213.74
112,3/31/2023,3444275.98


In [ ]:
# change date to datetime object
eu_gdp_per_capita['Date'] = pd.to_datetime(eu_gdp_per_capita['Date'])
eu_gdp_per_capita.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                114 non-null    datetime64[ns]
 1   EUR GDP Per Capita  114 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.9 KB


In [ ]:
# add to consolidated dataframe
eur_usd_df = eur_usd_df.merge(eu_gdp_per_capita, how='left', on='Date')

In [ ]:
# forward fill EU GDP Per Capita values
eur_usd_df['EUR GDP Per Capita'] = eur_usd_df['EUR GDP Per Capita'].ffill()
eur_usd_df

,Date,Open,High,Low,Close,Adj Close,EUR I/R,US Fed Rate,US Headline CPI,US Core CPI,EUR Headline CPI,EUR Core CPI,US GDP Per Capita,EUR GDP Per Capita
0,2010-01-01,1.432706,1.440196,1.432706,1.438994,1.438994,1.75,0.095714,217.488,220.633,92.32,93.52,47797.0,NaN
1,2010-01-02,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488,220.633,92.32,93.52,47797.0,NaN
2,2010-01-03,NaN,NaN,NaN,NaN,NaN,1.75,0.095714,217.488,220.633,92.32,93.52,47797.0,NaN
3,2010-01-04,1.431004,1.445191,1.426208,1.442398,1.442398,1.75,0.095714,217.488,220.633,92.32,93.52,47797.0,NaN
4,2010-01-05,1.442710,1.448310,1.435194,1.436596,1.436596,1.75,0.095714,217.488,220.633,92.32,93.52,47797.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4924,2023-06-26,1.090667,1.092180,1.088815,1.090667,1.090667,4.25,5.071429,303.841,308.309,123.15,119.48,80781.0,3444275.98
4925,2023-06-27,1.091048,1.097610,1.090322,1.091048,1.091048,4.25,5.071429,303.841,308.309,123.15,119.48,80781.0,3444275.98
4926,2023-06-28,1.095722,1.096251,1.089966,1.095722,1.095722,4.25,5.071429,303.841,308.309,123.15,119.48,80781.0,3444275.98
4927,2023-06-29,1.091584,1.094068,1.086236,1.091584,1.091584,4.25,5.071429,303.841,308.309,123.15,119.48,80781.0,3444275.98


# Current-account Debt

Current-account debt: Measures import and exports of goods <br>
Formula = (X-M) + NI + NT <br>

X - M = Exports - Imports <br>
NI = Net Income from foreign countries <br>
NT = Net transfers (government transfers)

# Terms of Trade

Terms of Trade: ratio of an index of a country's export prices to an index of its import prices

# Government Debt

# Unemployment rates

## US Unemployment Rates

In [ ]:
# read csv file
us_unemployment_rates = pd.read()

AttributeError: ignored

In [ ]:
# understand dataframe
us_unemployment_rates.info()

## EU Unemployment Rates

# Create CSV File

In [ ]:
# drop null values
eur_usd_df.dropna().reset_index(drop=True)

In [ ]:
# check for missing values
eur_usd_df.dropna().reset_index(drop=True).info()

In [ ]:
# create consolidated csv file
eur_usd_df.dropna().reset_index(drop=True).to_csv('EUR_USD Consolidated Data.csv', index=False)